In [1]:
!pip install pandas
!pip install scikeras[tensorflow]

In [2]:
!chmod +x /kaggle/input/classification2/download_cl2.sh
!source /kaggle/input/classification2/download_cl2.sh

chmod: impossível aceder a '/kaggle/input/classification2/download_cl2.sh': Ficheiro ou pasta inexistente
/bin/bash: linha 1: /kaggle/input/classification2/download_cl2.sh: Ficheiro ou pasta inexistente


In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as  plt
import gc
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2024-04-16 22:33:04.375575: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def alter_data(X):
    X = X.values
    turn = X[:,64]
    black_to_play = np.where(turn == -1)[0]
    # Multiply the values from indexes 0 to 63 by -1 where index 64 is 0
    X[black_to_play, :64] *= -1
    X = X[:, 0:64].reshape((X.shape[0], 8 , 8))
    return X

In [5]:
def load_n_datasets(n,foldername):
    all_datasets = []
    filenames = os.listdir(foldername)
    for i in range(n):
        filename = filenames[i]
        if filename.endswith('.csv'):
            dataset = pd.read_csv(os.path.join(foldername, filename))
            all_datasets.append(dataset)
    combined_dataset = pd.concat(all_datasets, ignore_index=True)
    
    X =  combined_dataset.iloc[:,0:-1]
    y =  combined_dataset["Evaluation"].astype(int)
    combined_dataset = None
    dataset = None
    all_datasets = None
    return X,y

In [6]:
def bit_map(X):
    channels = np.zeros((X.shape[0],8, 8, 12))  # 12 channels for 6 types of pieces for each player
    # Generate separate channels for each player
    for player in range(2):  # 0 for white pieces, 1 for black pieces
        if player ==0:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = X == (piece_type + 1)
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
        else:
            for piece_type in range(6):  # 6 types of pieces
                piece_mask = -1*(X == (-piece_type - 1))
                channels[:, :, :, player * 6 + piece_type] = piece_mask.astype(np.float32)
    X = None
    return channels

In [7]:
n_datasets = 1
X_chess_data,y_chess_data = load_n_datasets(n_datasets,'../classification_data2')

#X_chess_data, _, y_chess_data, _ = train_test_split(X_chess_data, y_chess_data, test_size=0.01, random_state=42, stratify=y_chess_data)

print(type(X_chess_data))
print(X_chess_data.shape)

X = alter_data(X_chess_data)
X = bit_map(X)
y = to_categorical(y_chess_data)

<class 'pandas.core.frame.DataFrame'>
(996771, 69)


In [8]:
X_chess_data = None
y_chess_data = None
gc.collect()

4

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
#80% training, 10% validation and 10% testing.

: 

In [ ]:
X = None
y = None
gc.collect()

In [ ]:
import tensorflow
from keras.models import Sequential 
from keras.layers import Conv2D, Flatten, BatchNormalization, Dense, Dropout
from keras.activations import elu
from keras.optimizers import SGD,Adam

tensorflow.random.set_seed(101)

In [ ]:
def create_model_mlp1(input_shape,num_classes):
    mlp_model = Sequential()

    # Add hidden layers with ReLU activation and dropout
    mlp_model.add(Dense(1048, activation='relu', input_shape=input_shape))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(500, activation='relu'))
    mlp_model.add(Dropout(0.2))
    mlp_model.add(Dense(50, activation='relu'))
    mlp_model.add(Dropout(0.2))

    # Layer to ensure the output matches the labels in shape
    mlp_model.add(Flatten())
    
    # Add output layer with softmax activation
    mlp_model.add(Dense(num_classes, activation='softmax'))

    # Compile model with Adam optimizer
    adam = Adam(learning_rate=0.001, beta_1=0.90, beta_2=0.99, epsilon=1e-8)
    mlp_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    mlp_model.summary()
    return mlp_model

In [ ]:
def create_model_cnn(input_shape,num_classes):
    cnn_model = Sequential()

    # Add convolutional layers with ELU activation and batch normalization
    cnn_model.add(Conv2D(20, (5, 5), activation=elu, input_shape=input_shape, padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Dropout(0.3))
    
    cnn_model.add(Conv2D(50, (3, 3), activation=elu, padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(Dropout(0.3))
    
    # Add fully connected layer
    cnn_model.add(Flatten())
    cnn_model.add(Dense(500, activation=elu))
    cnn_model.add(Dropout(0.3))
    
    # Add output layer with softmax activation
    cnn_model.add(Dense(num_classes, activation='softmax'))

    # Compile model with SGD optimizer
    sgd = SGD(learning_rate=0.01, nesterov=False)
    cnn_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    cnn_model.summary()
    return cnn_model

In [ ]:
def run_model(X_train,y_train,X_val,y_val,batch_size,epochs):
    
    input_shape = X_train.shape[1:]
    num_classes = 15
    print(input_shape)
    #model = create_model_mlp1(input_shape,num_classes)
    model = create_model_cnn(input_shape,num_classes)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    return history,model

In [ ]:
from sklearn.model_selection import StratifiedKFold,cross_val_score,KFold,cross_validate

In [ ]:
input_shape = X_train.shape[1:]
num_classes = 15

batch_size = 128
epochs = 8
num_splits = 5

# Create your model
model = create_model_cnn(input_shape,num_classes)

# Split data into 5 folds
kfold = KFold(n_splits=num_splits, shuffle=True, random_state=42)

# Perform cross-validation
results = cross_validate(model, X, y, cv=kfold, scoring='accuracy', return_estimator=True)

# Access scores for each fold
print("Scores for each fold:", results['test_score'])

# Access trained model objects for each fold
trained_models = results['estimator']
for i, model in enumerate(trained_models):
    print("Model for fold", i+1, ":", model)

In [ ]:
from sklearn.metrics import accuracy_score

loss,acc = model.evaluate(X_test,y_test)

print("Loss:",loss)
print("Accuracy:",acc)

In [ ]:
epochs_range = range(1, epochs + 1)

plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig('loss_cnn_alter_data')
plt.show()

In [ ]:
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.savefig('acc_cnn_alter_data')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

predicted = np.argmax(model.predict(X_test),axis=1)

actual = np.argmax(y_test,axis=1)

In [ ]:
confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [i for i in range(15)])

cm_display.plot()
plt.show() 

In [ ]:
y_df = pd.DataFrame(y_test)
class_counts = y_df.value_counts()
class_counts = class_counts.sort_index()

print(class_counts)